In [2]:
import numpy as np
from qiskit import *
#This is the quanutm circuit with two bits of information
backend = Aer.get_backend('statevector_simulator')
circ = QuantumCircuit(4,4)

#Prepare the second qbit in a superposition |0> + |1>
initial_state = [0,1]
circ.initialize(initial_state, 1)
circ.h(1)
#Project to the x+- states
circ.h(2) #Hadamard Gate
#0->1 state for 3rd qbit
initial_state = [0,1]
circ.initialize(initial_state, 3)
circ.h(3) #Hadamard Gate

T = 150 #Total Time
n = 2000 #Iterations
dt = T/n
for t in range(n+1):
    A = 1-t/T
    B = t/T
    
    #Term attached to A(t)
    evoloution_operator = QuantumCircuit(4,name='ABA')
    evoloution_operator.rz(A*np.pi*dt,2)
    evoloution_operator.rz(A*np.pi*dt,3)
    evoloution_operator.rzz(A*np.pi*dt,0,2)
    evoloution_operator.rzz(A*np.pi*dt,1,2)
    evoloution_operator.rzz(A*np.pi*dt,0,3)
    evoloution_operator.rzz(A*np.pi*dt,1,3)
    
    #Term attached to B(t)
    evoloution_operator.rx(B*np.pi*2*dt,2)
    evoloution_operator.rx(B*np.pi*2*dt,3)
    
    #Term attached to A(t)
    evoloution_operator.rz(A*np.pi*dt,2)
    evoloution_operator.rz(A*np.pi*dt,3)
    evoloution_operator.rzz(A*np.pi*dt,0,2)
    evoloution_operator.rzz(A*np.pi*dt,1,2)
    evoloution_operator.rzz(A*np.pi*dt,0,3)
    evoloution_operator.rzz(A*np.pi*dt,1,3)
    
    evoloution_operator.to_gate()
    circ.append(evoloution_operator,[0,1,2,3])
    if (t == 2000):
        break
#Project to the z01 states
circ.h(1)
circ.h(2) #Hadamard Gate
circ.h(3) #Hadamard Gate
#print(circ)

In [ ]:
circ = circ.reverse_bits() #Key to reading the bits from q0->q3 rather than q3->q0
circ.measure([0,1,2,3],[0,1,2,3])
job=execute(circ,backend,shots=1000)
result=job.result()
result.get_counts()